In [13]:
#initialize pytrends and import statements
import statistics
import pandas as pd
from pytrends.request import TrendReq
from time import sleep
pytrends = TrendReq(hl='en-US', tz=240)

In [14]:
#Create the method to extract they keyword and data for the given timeframe

def gatherDataForKeyWord(wordlist, stateCode, timeperiod):
    pytrends.build_payload(wordlist, cat=0, timeframe=timeperiod, geo=stateCode, gprop='')
    sleep(1.5)
    return pytrends.interest_over_time()



#Method to convert the data from pytrends df to project structure
def convertPytrends(df, keywordCols, wordDf):
    for col in keywordCols:
        colAsList = df[col].to_dict()
        yearsValues = extractYearValues(colAsList, wordDf)


#Method to extract the min max mean and median from the pytrends col
def extractYearValues(colAsDict, wordDF):
    for key in colAsDict.keys():
        currentKey = key.year
        if currentKey in wordDF:
            wordDF[currentKey].append(colAsDict[key])
        else:
            wordDF.update({currentKey : [colAsDict[key]]})

    return wordDF

def getTrendData(yearDict):
    yearVals = {}
    for year in yearDict.keys():
        yearMin = min(yearDict[year])
        yearMax = max(yearDict[year])
        yearMean = statistics.mean(yearDict[year])
        yearMedian = statistics.median(yearDict[year])
        yearVals.update({year : {'min' : yearMin, 'max' : yearMax, 'mean' : yearMean, 'median' : yearMedian}})

    return yearVals

In [15]:
colKeyWordsDiagnoses = ['depression', 'anxiety', 'ADHD', 'bipolar', 'PTSD']
colKeyWordsPhrases = ['psychiatrists near me', 'psychologist near me', 'therapist near me', 
               'mental hospital']
timeframes = ['2013-01-01 2024-12-31'] #,'2016-01-01 2020-12-31', '2021-01-01 today']
resultDict = {}
stateCodes = [ 'US-AL', 'US-AK', 'US-AZ', 'US-AR', 'US-CA', 'US-CO', 'US-CT', 'US-DC', 'US-DE', 
              'US-FL', 'US-GA', 'US-HI', 'US-ID', 'US-IL', 'US-IN', 'US-IA', 'US-KS', 'US-KY', 
              'US-LA', 'US-ME', 'US-MD', 'US-MA', 'US-MI', 'US-MN', 'US-MS', 'US-MO', 'US-MT', 
              'US-NE', 'US-NV', 'US-NH', 'US-NJ', 'US-NM', 'US-NY', 'US-NC', 'US-ND', 'US-OH', 
               'US-OK', 'US-OR', 'US-PA', 'US-RI', 'US-SC', 'US-SD', 'US-TN', 'US-TX', 'US-UT', 
               'US-VT', 'US-VA', 'US-WA', 'US-WV', 'US-WI', 'US-WY']
for state in stateCodes:
    stateDict = {}
    wordDf = {}
    for time in timeframes:
        print(f"State : {state}, Time : {time}")
        pyTrendsDf = gatherDataForKeyWord(colKeyWordsDiagnoses, state, time)
        for word in colKeyWordsDiagnoses:
            wordDf = {}
            convertPytrends(pyTrendsDf, [word], wordDf)
            yearTrendData = getTrendData(wordDf)
            stateDict.update({word : yearTrendData})

    for time in timeframes:
        pyTrendsDf = gatherDataForKeyWord(colKeyWordsPhrases, state, time)
        for word in colKeyWordsPhrases:
            wordDf = {}
            convertPytrends(pyTrendsDf, [word], wordDf)
            yearTrendData = getTrendData(wordDf)
            stateDict.update({word : yearTrendData})
    resultDict.update({state : stateDict})
print(resultDict)

State : US-AL, Time : 2013-01-01 2024-12-31


c:\Users\johnb\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
c:\Users\johnb\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


State : US-AK, Time : 2013-01-01 2024-12-31


c:\Users\johnb\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
c:\Users\johnb\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


State : US-AZ, Time : 2013-01-01 2024-12-31


...

c:\Users\johnb\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
c:\Users\johnb\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


State : US-WY, Time : 2013-01-01 2024-12-31


c:\Users\johnb\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


{'US-AL': {'depression': {2013: {'min': 45, 'max': 77, 'mean': 59, 'median': 59.5}, 2014: {'min': 47, 'max': 78, 'mean': 64.75, 'median': 67.5}, 2015: {'min': 50, 'max': 85, 'mean': 70.41666666666667, 'median': 71.5}, 2016: {'min': 49, 'max': 82, 'mean': 67.91666666666667, 'median': 69.5}, 2017: {'min': 55, 'max': 85, 'mean': 69.75, 'median': 71.0}, 2018: {'min': 62, 'max': 85, 'mean': 73, 'median': 73.0}, 2019: {'min': 55, 'max': 85, 'mean': 71.16666666666667, 'median': 72.0}, 2020: {'min': 58, 'max': 76, 'mean': 65.83333333333333, 'median': 65.5}, 2021: {'min': 53, 'max': 66, 'mean': 60.666666666666664, 'median': 62.0}, 2022: {'min': 54, 'max': 80, 'mean': 64.58333333333333, 'median': 63.5}, 2023: {'min': 49, 'max': 72, 'mean': 60, 'median': 58.0}, 2024: {'min': 46, 'max': 68, 'mean': 55.90909090909091, 'median': 56}}, 'anxiety': {2013: {'min': 41, 'max': 52, 'mean': 47.25, 'median': 47.5}, 2014: {'min': 43, 'max': 54, 'mean': 49.083333333333336, 'median': 49.5}, 2015: {'min': 52, 'm

c:\Users\johnb\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


In [17]:
rows = []
for state, conditions in resultDict.items():
    for condition, years in conditions.items():
        for year, stats in years.items():
            key = f'{state}_{year}'
            row = {'key': key, 'condition': condition}
            row.update(stats)
            rows.append(row)

# Creating the DataFrame
df = pd.DataFrame(rows)

# Pivoting the DataFrame
df_pivot = df.pivot_table(index='key', columns='condition', values=['min', 'max', 'mean', 'median'])

# Flatten the MultiIndex columns
df_pivot.columns = ['_'.join([str(i) for i in col]).strip() for col in df_pivot.columns.values]

# Reset the index
df_pivot.reset_index(inplace=True)

# Save to CSV
df_pivot.to_csv("../data/pytrends_data.csv")
